In [1]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pytorch-lightning

In [2]:
import pytorch_lightning as pl
import pandas as pd
import cv2
import os 
from torch import nn
from torch.utils.data import Dataset ,DataLoader, random_split
import numpy as np
import torch
from sklearn.model_selection import train_test_split 
from torchvision import transforms, datasets, models
import matplotlib.pyplot as plt
import torchmetrics
from torchmetrics.functional import accuracy
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning.callbacks.progress import TQDMProgressBar
from pytorch_lightning.loggers import CSVLogger
from torchvision.utils import make_grid
import math
import torch.nn.functional as F

In [3]:
pl.seed_everything(7)
PATH_DATASETS = os.environ.get("PATH_DATASETS", "/content/drive/MyDrive/Datasets")
BATCH_SIZE = 256 if torch.cuda.is_available() else 64
CLASS_SIZE = 10
VAL_SIZE = 10000

Global seed set to 7


In [4]:
class LitDataModule(pl.LightningDataModule):
  def __init__(self, 
               data_dir: str=PATH_DATASETS,
               batch_size: int=BATCH_SIZE,
               num_class: int=CLASS_SIZE,
               val_size: int=VAL_SIZE):
    super().__init__()
    self.data_dir = data_dir
    self.batch_size = batch_size
    self.num_class = num_class
    self.val_size = val_size
    self.transform = {
        'train': transforms.Compose([
            transforms.Resize((224,224)),
            transforms.RandomHorizontalFlip(p=0.7),
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
        ]),

        'test': transforms.Compose([
            transforms.Resize((224,224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
        ])
    }

  def prepare_data(self):
    datasets.CIFAR10(root=self.data_dir, train=True, download=True)
    datasets.CIFAR10(root=self.data_dir, train=False, download=True)

  def setup(self, stage=None):
    if stage=='fit' or stage is None:
      cf_full = datasets.CIFAR10(root=self.data_dir, train=True, transform=self.transform['train'])
      train_size = len(cf_full) - self.val_size
      self.cf_train, self.cf_val = random_split(cf_full, [train_size, self.val_size])

    if stage=='test' or stage is None:
      self.cf_test = datasets.CIFAR10(root=self.data_dir, train=False, transform=self.transform['test'])

  def train_dataloader(self):
    return DataLoader(self.cf_train, batch_size=256, shuffle=True)

  def val_dataloader(self):
    return DataLoader(self.cf_val, batch_size=256, shuffle=False)

  def test_dataloader(self):
    return DataLoader(self.cf_test, batch_size=256, shuffle=False)

  def visualize(self):
    for images, _ in self.train_dataloader():
      print(f"images.shape: {images.shape}")
      plt.figure(figsize=(16,8))
      plt.axis("off")
      plt.imshow(make_grid(images, nrow=16).permute(1, 2, 0))
      break

In [5]:
class LitVGG16(pl.LightningModule):
  def __init__(self, input_shape, num_classes, lr=2e-4, transfer=False):
    super().__init__()

    self.save_hyperparameters()
    self.lr = lr
    self.dim = input_shape
    self.num_classes = num_classes

    # transfer learning if pre-trained true
    self.feature_extractor = models.vgg16(pretrained=transfer)

    if transfer:
      # layers are frozen by using eval()
      self.feature_extractor.eval()
      # freeze params
      for param in self.feature_extractor.parameters():
        param.requires_grad = False

    n_sizes = self._get_conv_output(input_shape)

    self.classifier = nn.Linear(n_sizes, num_classes)

  # returns the size of the output tensor going into the Linear layer from the conv block
  def _get_conv_output(self, shape):
    batch_size = 1
    tmp_input = torch.autograd.Variable(torch.rand(batch_size, *shape))

    output_feat = self._forward_features(tmp_input)
    n_size = output_feat.data.view(batch_size, -1).size(1)
    return n_size

  # returns the feature tensor from the conv block
  def _forward_features(self, x):
    x = self.feature_extractor(x)
    return x


  def forward(self, x):
    x = self._forward_features(x)
    x = x.view(x.size(0), -1)
    x = self.classifier(x)
    return x

  def training_step(self, batch, batch_idx):
    x, y = batch
    logits = self.forward(x)
    loss = nn.functional.cross_entropy(logits, y)
    self.log('train loss', loss)
    return loss

  def evaluate(self, batch, stage=None):
    x, y = batch
    logits = self.forward(x)
    loss = nn.functional.cross_entropy(logits, y)
    preds = torch.argmax(logits, dim=1)
    acc = accuracy(preds, y)

    if stage:
      self.log(f"{stage}_loss", loss, prog_bar=True)
      self.log(f"{stage}_acc", acc, prog_bar=True)

  def validation_step(self, batch, batch_idx):
    self.evaluate(batch, "val")

  def test_step(self, batch, batch_idx):
    self.evaluate(batch, "test")

  def configure_optimizers(self):
    return torch.optim.Adam(self.parameters(), lr=self.lr)

In [6]:
dm = LitDataModule()
dm.prepare_data()

Files already downloaded and verified
Files already downloaded and verified


In [7]:
dm.setup()

In [45]:
x = dm.cf_test[0]
x0 = x[0]

In [46]:
x1 = x[1]

In [49]:
x0.size()

torch.Size([3, 224, 224])

In [8]:
model = LitVGG16([3, 224, 224], CLASS_SIZE, transfer=False)
trainer = pl.Trainer(max_epochs=10,
                  accelerator="auto",
                  devices=1 if torch.cuda.is_available() else None,
                  callbacks=[LearningRateMonitor(logging_interval="step"),
                                TQDMProgressBar(refresh_rate=10)],
)
trainer.fit(model, dm)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Files already downloaded and verified
Files already downloaded and verified


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type   | Params
---------------------------------------------
0 | feature_extractor | VGG    | 138 M 
1 | classifier        | Linear | 10.0 K
---------------------------------------------
138 M     Trainable params
0         Non-trainable params
138 M     Total params
553.470   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

RuntimeError: ignored

In [57]:
trainer.test(model, dm)

Files already downloaded and verified
Files already downloaded and verified


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.8105999827384949
        test_loss           0.5389890074729919
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_acc': 0.8105999827384949, 'test_loss': 0.5389890074729919}]